In [67]:
from Bending.FullBending import FullBending
import numpy as np
import pandas as pd

In [68]:
# E [GPa]
# nucleo	4.22
# X	26.95
# SX	13.57
# Y	97.86
# SY	4.22

df_layers = pd.DataFrame({ 'E': [4.01,
                                 6.02, 
                                 4.01, 
                                 14.13, 
                                 4.01]}, index=['nucleo', 'X', 'SX', 'Y', 'SY'])

In [69]:
df_layers

,E
nucleo,4.01
X,6.02
SX,4.01
Y,14.13
SY,4.01


In [70]:
composition = [
        ["Y", "Y"],
        ["Y", "X"],
        ["Y", "SX"],
        ["Y", "SY"],
        ["Y","SX","SY"],
        ["Y","X", "SY"]
    ]


In [71]:
from validation.BendingExperimental import BendingExperimental

r = BendingExperimental()

df_stats = r["df_stats"]


In [72]:
df_stats["Modulo flexion (GPa)"]

,mean,std
Name,,
TFP1,13.201050,0.630858
TFP2,9.869501,0.780113
TFP3,9.222003,0.495965
TFP4,10.181815,0.770635
TFP5,6.143105,0.161637
TFP6,7.798060,0.334186


In [73]:

import os 
from copy import deepcopy
join = os.path.join

E_pred = []

if not os.path.exists("output"):
    os.mkdir("output")

params = {
            "geo" : {
                        "t_n" : 2.51,
                        "t_l" : 0.5,
                        "Lx"  : 30,
                    },
            "sim" : {
                        "disp" : 0.5,
                    },
        }


all_params = []

id_design = 0
E_n =   df_layers.loc["nucleo"].values[-1]*1e3

lyrs = df_stats["layers"].iloc[id_design]

E_l = df_layers.loc[lyrs]["E"].values*1e3 # MPa
name_mat = df_stats["layers"].iloc[id_design]
name = df_stats.index[id_design]
B    = df_stats["Ancho flexion (mm)"]["mean"].iloc[id_design]

params_loop = deepcopy(params)

B = 25
params_loop["geo"]["Ly"] = B
params_loop["sim"]["E_n"] = E_n
params_loop["sim"]["E_l"] = E_l
params_loop["sim"]["name_mat"] = name_mat
params_loop["output_folder"] = join("output")

print(params)
inp_f = FullBending(params_loop)
Eflex = params_loop["Eflex"]


{'geo': {'t_n': 2.51, 't_l': 0.5, 'Lx': 30}, 'sim': {'disp': 0.5}}
Running Calculix at:  output\sim
Output file:  c:\Users\djoroya\Documents\GitHub\CITISENS_Composite\scripts\T02_BendingTest\NumericalBasic\output\sim\out.txt
Command:
 mpiexec -n 4 c:\Users\djoroya\Documents\GitHub\CITISENS_Composite\.conda\Lib\site-packages\djccx\bin\ccx_dynamic.exe main
Error reading cvf file

pid:  13752 

Calculix finished



In [74]:
def E_flexion_an(E_n,E_l,t_n,t_l):

    t_t = t_n + 2*t_l

    yG = t_n*0.5 + t_l*0.5

    I_tot = (1/12)*t_t**3
    
    I_l   = (1/12)*t_l**3 + t_l*yG**2

    I_n   = (1/12)*t_n**3 


    Eflex = (E_n*I_n + 2*E_l*I_l)/(I_tot)

    return Eflex*1e-3 # GPa

In [75]:
t_n = params_loop["geo"]["t_n"]
t_l = params_loop["geo"]["t_l"]

Eflex_pred = E_flexion_an(E_n,
                          E_l[0],
                          t_n,
                          t_l)

In [76]:
Eflex_pred - params_loop["Eflex"]

print("Analitico: ", Eflex_pred)
print("Simulado: ", Eflex)


Analitico:  10.429330271928873
Simulado:  10.851424147848933
